In [72]:
import time

from trade_ibkr.model import OnPxDataUpdatedEventNoAccount
from trade_ibkr.obj import start_app_info
from trade_ibkr.utils import make_futures_contract


async def on_px_updated(e: OnPxDataUpdatedEventNoAccount):
    file_name = f"data-{e.contract.underSymbol}@{e.px_data.period_sec}.csv"
    e.px_data.dataframe.to_csv(file_name)
    print(f"Px Data saved to: {file_name}")


async def on_market_data_received():
    pass


def store_historical_data():
    app, _ = start_app_info(is_demo=True)

    contract_mnq = make_futures_contract("MNQH2", "GLOBEX")
    contract_mym = make_futures_contract("MYM  MAR 22", "ECBOT")

    px_data_req_ids: list[int] = [
        req_id_same_contract for req_ids_cross_contract
        in [
            app.get_px_data_keep_update(
                contract=contract_mnq,
                duration="10 D",
                bar_sizes=["1 min"],
                period_secs=[60],
                on_px_data_updated=on_px_updated,
                on_market_data_received=on_market_data_received,
            ),
            app.get_px_data_keep_update(
                contract=contract_mym,
                duration="10 D",
                bar_sizes=["1 min"],
                period_secs=[60],
                on_px_data_updated=on_px_updated,
                on_market_data_received=on_market_data_received,
            ),
        ]
        for req_id_same_contract in req_ids_cross_contract
    ]

    while not app.is_all_px_data_ready(px_data_req_ids):
        time.sleep(0.5)
        print("Loading data...")

    app.disconnect()
    print("Completed.")

# store_historical_data()

In [111]:
import pandas as pd

df_mnq = pd.read_csv("archive/futures/NQ/20220220-20220304-1.csv")
df_mnq = df_mnq[["date", "close"]]
df_mym = pd.read_csv("archive/futures/YM/20220220-20220304-1.csv")
df_mym = df_mym[["date", "close"]]

In [112]:

df = pd.merge(df_mnq, df_mym, on="date", suffixes=("_mnq", "_mym"))
df.index = pd.to_datetime(df["date"])
df.drop(columns=["date"], inplace=True)
df

,close_mnq,close_mym
date,,
2022-02-20 17:00:00,13932.75,33921.0
2022-02-20 17:01:00,13859.50,33907.0
2022-02-20 17:02:00,13884.75,33923.0
2022-02-20 17:03:00,13875.50,33907.0
2022-02-20 17:04:00,13873.50,33915.0
...,...,...
2022-03-04 15:55:00,13800.50,33524.0
2022-03-04 15:56:00,13802.00,33524.0
2022-03-04 15:57:00,13800.00,33526.0


In [113]:
import numpy as np
import talib
from talib import MA_Type

df["px_diff"] = np.log(df["close_mym"].divide(df["close_mnq"]))
upper, mid, lower = talib.BBANDS(df["px_diff"], timeperiod=10, nbdevup=2, nbdevdn=2, matype=MA_Type.SMA)
df["px_diff_upper"] = upper
df["px_diff_mid"] = mid
df["px_diff_lower"] = lower
df

,close_mnq,close_mym,px_diff,px_diff_upper,px_diff_mid,px_diff_lower
date,,,,,,
2022-02-20 17:00:00,13932.75,33921.0,0.889792,NaN,NaN,NaN
2022-02-20 17:01:00,13859.50,33907.0,0.894651,NaN,NaN,NaN
2022-02-20 17:02:00,13884.75,33923.0,0.893302,NaN,NaN,NaN
2022-02-20 17:03:00,13875.50,33907.0,0.893497,NaN,NaN,NaN
2022-02-20 17:04:00,13873.50,33915.0,0.893877,NaN,NaN,NaN
...,...,...,...,...,...,...
2022-03-04 15:55:00,13800.50,33524.0,0.887557,0.888013,0.887704,0.887394
2022-03-04 15:56:00,13802.00,33524.0,0.887448,0.887962,0.887656,0.887350
2022-03-04 15:57:00,13800.00,33526.0,0.887653,0.887824,0.887621,0.887419


In [114]:
df_selected = df.between_time("03:00", "07:15")
df_selected

,close_mnq,close_mym,px_diff,px_diff_upper,px_diff_mid,px_diff_lower
date,,,,,,
2022-02-21 02:00:00,14095.50,34289.0,0.888969,0.889364,0.888952,0.888540
2022-02-21 02:01:00,14086.00,34275.0,0.889235,0.889392,0.889013,0.888634
2022-02-21 02:02:00,14090.00,34278.0,0.889038,0.889378,0.889042,0.888706
2022-02-21 02:03:00,14091.25,34270.0,0.888716,0.889379,0.889041,0.888704
2022-02-21 02:04:00,14092.50,34265.0,0.888482,0.889411,0.888965,0.888519
...,...,...,...,...,...,...
2022-03-04 06:26:00,13893.75,33387.0,0.876728,0.877050,0.876695,0.876341
2022-03-04 06:27:00,13891.25,33383.0,0.876788,0.877061,0.876703,0.876344
2022-03-04 06:28:00,13877.75,33361.0,0.877101,0.877156,0.876729,0.876303


In [115]:
from pandas import DataFrame

executions = []
current_pnl = 0

for dt, row in df_selected.iterrows():
    last = executions[-1] if executions else None

    if not last or last["position"] == "close":
        if row["px_diff"] > row["px_diff_upper"]:
            executions.append({
                "buy": "MNQ",
                "sell": "MYM",
                "entry_t": dt,
                "entry_mnq": row["close_mnq"],
                "entry_mym": row["close_mym"],
                "exit_mnq": None,
                "exit_mym": None,
                "exit_t": None,
                "position": "open",
            })

        if row["px_diff"] < row["px_diff_lower"]:
            executions.append({
                "buy": "MYM",
                "sell": "MNQ",
                "entry_t": dt,
                "entry_mnq": row["close_mnq"],
                "entry_mym": row["close_mym"],
                "exit_mnq": None,
                "exit_mym": None,
                "exit_t": None,
                "position": "open",
            })

        continue
    elif (last["buy"] == "MNQ" and row["px_diff"] < row["px_diff_mid"]) or (last["buy"] == "MYM" and row["px_diff"] > row["px_diff_mid"]):
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }
        current_pnl = 0
        
    if current_pnl < -100:
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }

df_exec = DataFrame(executions)
df_exec["px_side_mnq"] = (df_exec["exit_mnq"] - df_exec["entry_mnq"]) * np.where(df_exec["buy"] == "MNQ", 1, -1)
df_exec["px_side_mym"] = (df_exec["exit_mym"] - df_exec["entry_mym"]) * np.where(df_exec["buy"] == "MYM", 1, -1)
df_exec["pnl_mnq"] = df_exec["px_side_mnq"] * 1 * 2
df_exec["pnl_mym"] = df_exec["px_side_mym"] * 3 * 0.5
df_exec["pnl_single"] = df_exec["pnl_mnq"] + df_exec["pnl_mym"] - 0.52 * 4
df_exec["pnl_cum"] = df_exec["pnl_single"].cumsum()
df_exec

,buy,sell,entry_t,entry_mnq,entry_mym,exit_mnq,exit_mym,exit_t,position,px_side_mnq,px_side_mym,pnl_mnq,pnl_mym,pnl_single,pnl_cum
0,MYM,MNQ,2022-02-21 02:04:00,14092.50,34265.0,14092.00,34261.0,2022-02-21 02:13:00,close,0.50,-4.0,1.0,-6.0,-7.08,-7.08
1,MNQ,MYM,2022-02-21 02:27:00,14094.25,34274.0,14105.25,34287.0,2022-02-21 02:32:00,close,11.00,-13.0,22.0,-19.5,0.42,-6.66
2,MNQ,MYM,2022-02-21 02:35:00,14059.00,34217.0,14045.00,34193.0,2022-02-21 02:46:00,close,-14.00,24.0,-28.0,36.0,5.92,-0.74
3,MNQ,MYM,2022-02-21 02:55:00,14035.50,34197.0,14037.25,34189.0,2022-02-21 02:59:00,close,1.75,8.0,3.5,12.0,13.42,12.68
4,MYM,MNQ,2022-02-21 03:24:00,14011.50,34098.0,14016.75,34128.0,2022-02-21 03:26:00,close,-5.25,30.0,-10.5,45.0,32.42,45.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,MYM,MNQ,2022-03-04 05:04:00,13926.50,33462.0,13922.00,33460.0,2022-03-04 05:12:00,close,4.50,-2.0,9.0,-3.0,3.92,821.74
147,MNQ,MYM,2022-03-04 05:29:00,13926.50,33491.0,13924.75,33475.0,2022-03-04 05:35:00,close,-1.75,16.0,-3.5,24.0,18.42,840.16
148,MYM,MNQ,2022-03-04 05:45:00,13923.50,33458.0,13929.25,33468.0,2022-03-04 05:57:00,close,-5.75,10.0,-11.5,15.0,1.42,841.58
149,MYM,MNQ,2022-03-04 06:05:00,13936.00,33458.0,13910.00,33410.0,2022-03-04 06:09:00,close,26.00,-48.0,52.0,-72.0,-22.08,819.50


In [116]:
import plotly.express as px

fig = px.line(df_exec, x=df_exec.index, y="pnl_cum")
fig.show()